In [1]:
import numpy as np

In [3]:
# np.nan_to_num()
# numpy.nan_to_num(x,copy=True,nan=0.0,posinf=None,neginf=None)

# 유효하지 않은 값으로 배열 생성
arr = np.array([1, np.nan, 3, np.inf])
print(arr)

# 유효하지 않은 값 교체
arr = np.nan_to_num(arr)
print(arr)

[ 1. nan  3. inf]
[1.00000000e+000 0.00000000e+000 3.00000000e+000 1.79769313e+308]


## 7.4. RNN
* IMDB 데이터셋: 영화 리뷰에 대한 데이터 5만 개로 구성
* 훈련 data: 2만 5000개 / 테스트 data: 2만 5000개 (50%씩 긍정, 부정 리뷰)
* imdb.load_data() 메서드로 바로 내려 받아 사용 가능

### 7.4.1. RNN 셀 구현

In [1]:
# (코드 7-3) 라이브러리 호출
import torch
import torchtext # NLP 분야에서 사용되는 데이터로더
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import time

# import torchtext 때문에 오류 발생했음 >> 해결 방법(prompt에 코드 입력): pip install torch==2.0.0+cpu torchvision==0.15.1+cpu torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cpu

In [ ]:
# torchtext.legacy 가 자꾸 에러 떠러 버전을 강제로 prompt에서 조정해줌
# !pip install torchtext==0.10.1 --user

In [2]:
# (코드 7-4) 데이터 전처리
start=time.time()
TEXT = torchtext.legacy.data.Field(lower=True, fix_length=200, batch_first=False)
LABEL = torchtext.legacy.data.Field(sequential=False) # 이 레이블은 범주(긍정/부정)만 가지기 때문에 sequential=False 가 됨 (시계열일 경우 True)

In [3]:
# (코드 7-5) 데이터셋 준비
from torchtext.legacy import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


c:\Users\USER\Desktop\BOAZ\23-2.02. 멘토멘티 스터디\MentorMentee_Study_Code\.data\imdb\aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [03:28<00:00, 403kB/s] 


In [16]:
# (코드 7-6) 훈련 데이터셋 내용 확인
print(vars(train_data.examples[0]))
print()
print(vars(train_data.examples[0])['label']) # pos: positive
print(vars(train_data.examples[20000])['label']) # neg: negative

{'text': ['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy.', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life,', 'such', 'as', '"teachers".', 'my', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'bromwell', "high's", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', '"teachers".', 'the', 'scramble', 'to', 'survive', 'financially,', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', "teachers'", 'pomp,', 'the', 'pettiness', 'of', 'the', 'whole', 'situation,', 'all', 'remind', 'me', 'of', 'the', 'schools', 'i', 'knew', 'and', 'their', 'students.', 'when', 'i', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school,', 'i', 'immediately', 'recalled', '.........', 'at', '..........', 'high.', 'a', 'classic', 'line:', 'inspector:', "i'm", 'here', 'to', 'sack', 'one', 'of', '

In [17]:
# (코드 7-7) 데이터셋 전처리 적용: 공백처리, 불필요한 문자 제거
import string

for example in train_data.examples:
    text = [x.lower() for x in vars(example)['text']] # text의 내용을 소문자로 변경 >> 이를 text 변수에 저장
    text = [x.replace("<br","") for x in text] # 위의 text 변수에서 "<br"을 "" (공백)으로 변경 >> text 변수 업데이트
    text = [''.join(c for c in s if c not in string.punctuation) for s in text] # 구두점 제거
    text = [s for s in text if s] # 공백 제거
    vars(example)['text'] = text # 위에서 새롭게 정의한 text 변수를 훈련 데이터셋의 text 딕셔너리의 key 값의 value값으로 대입

In [18]:
# (코드 7-7.2) 코드 7-6과 결과 값 비교 >> 데이터셋 전처리 깔끔하게 됨
print(vars(train_data.examples[0]))

{'text': ['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', 'such', 'as', 'teachers', 'my', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'bromwell', 'highs', 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', 'teachers', 'the', 'scramble', 'to', 'survive', 'financially', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', 'teachers', 'pomp', 'the', 'pettiness', 'of', 'the', 'whole', 'situation', 'all', 'remind', 'me', 'of', 'the', 'schools', 'i', 'knew', 'and', 'their', 'students', 'when', 'i', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school', 'i', 'immediately', 'recalled', 'at', 'high', 'a', 'classic', 'line', 'inspector', 'im', 'here', 'to', 'sack', 'one', 'of', 'your', 'teachers', 'student', 'welcome', 'to',

In [19]:
# (코드 7-8) 훈련 / 검증 데이터셋 분리
import random
train_data, valid_data = train_data.split(random_state=random.seed(0), split_ratio=0.8)
# seed 값을 통해 동일한 코드를 여러 번 수행하더라도 동일한 값의 데이터 반환
# 훈련: 검증 = 8:2

In [20]:
# (코드 7-9) 데이터셋 개수 확인
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 20000
Number of validation examples: 5000
Number of testing examples: 25000


In [21]:
# (코드 7-10) 단어 집합 만들기
# 아래 코드에서 vectors=None 으로, 임베딩 처리 부분에 대해 정의하지 않음 >> (코드 7-13)에서 진행할 예정
TEXT.build_vocab(train_data, max_size=10000, min_freq=10, vectors=None)
LABEL.build_vocab(train_data)

print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")
# 실행 결과에서 LABEL은 긍정, 부정 두 개의 값만 나와야 하는데, 세 개가 있다고 출력됨 >> (코드 7-11)에서 확인

Unique tokens in TEXT vocabulary: 10002
Unique tokens in LABEL vocabulary: 3


In [22]:
# (코드 7-11) 테스트 데이터셋의 단어 집합 확인
print(LABEL.vocab.stoi)

# <unk>, pos, neg 로 구성되어 있음
# <unk> : 사전에 없는 단어 의미

defaultdict(<bound method Vocab._default_unk_index of <torchtext.legacy.vocab.Vocab object at 0x000001786FCB5790>>, {'<unk>': 0, 'pos': 1, 'neg': 2})


In [23]:
# (코드 7-12) 데이터셋 메모리로 가져오기 >> BucketIterator() 이용
BATCH_SIZE = 64
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

embeding_dim = 100
hidden_size = 300

train_iterator, valid_iterator, test_iterator = torchtext.legacy.data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [24]:
# (코드 7-13) 워드 임베딩(nn.Embedding()) 및 RNN 셀 정의
class RNNCell_Encoder(nn.Module):
    def __init__(self, input_dim, hidden_size):
        super(RNNCell_Encoder, self).__init__()
        self.rnn = nn.RNNCell(input_dim, hidden_size)

    def forward(self, inputs):
        bz = inputs.shape[1]
        ht = torch.zeros((bz, hidden_size)).to(device)

        for word in inputs:
            ht = self.rnn(word, ht)
        return ht

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.em = nn.Embedding(len(TEXT.vocab.stoi), embeding_dim)
        self.rnn = RNNCell_Encoder(embeding_dim, hidden_size)
        self.fc1 = nn.Linear(hidden_size, 256)
        self.fc2 = nn.Linear(256, 3)

    def forward(self, x):
        x = self.em(x)
        x = self.rnn(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [25]:
# (코드 7-14) 옵티마이저와 손실 함수 정의
model = Net() # model 변수명으로 모델 객체화
model.to(device)

loss_fn = nn.CrossEntropyLoss() # 다중 분류에 사용됨
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [26]:
# (코드 7-15) 모델 학습을 위한 함수 생성
def training(epoch, model, trainloader, validloader):
    correct = 0
    total = 0
    running_loss = 0

    model.train()
    for b in trainloader:
        x, y = b.text, b.label # trainloader에서 text와 label을 꺼내옴
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        loss = loss_fn(y_pred, y) # CrossEntropyLoss 손실 함수를 이용해서 오차 계산
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            y_pred = torch.argmax(y_pred, dim=1)
            correct += (y_pred == y).sum().item()
            total += y.size(0)
            running_loss += loss.item()
    epoch_loss = running_loss / len(trainloader.dataset)
    epoch_acc = correct / total

    valid_correct = 0
    valid_total = 0
    valid_running_loss = 0

    model.eval()
    with torch.no_grad():
        for b in validloader:
            x, y = b.text, b.label
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            y_pred = torch.argmax(y_pred, dim=1)
            valid_correct += (y_pred == y).sum().item()
            valid_total += y.size(0)
            valid_running_loss += loss.item()

    epoch_valid_loss = valid_running_loss / len(validloader.dataset)
    epoch_valid_acc = valid_correct / valid_total

    print('epoch: ', epoch,
          'loss: ', round(epoch_loss, 3),
          'accuracy:', round(epoch_acc, 3),
          'valid_loss: ', round(epoch_valid_loss, 3),
          'valid_accuracy:', round(epoch_valid_acc, 3)
          )
    return epoch_loss, epoch_acc, epoch_valid_loss, epoch_valid_acc

In [27]:
# (코드 7-16) 훈련과 검증 데이터셋을 이용한 모델 학습
epochs = 5
train_loss = []
train_acc = []
valid_loss = []
valid_acc = []

for epoch in range(epochs):
    epoch_loss, epoch_acc, epoch_valid_loss, epoch_valid_acc = training(epoch, model, train_iterator, valid_iterator)
    train_loss.append(epoch_loss) # 훈련 데이터셋을 모델에 적용했을 때의 '오차'
    train_acc.append(epoch_acc) # 훈련 데이터셋을 모델에 적용했을 때의 '정확도'
    valid_loss.append(epoch_valid_loss) # 검증 데이터셋을 모델에 적용했을 때의 '오차'
    valid_acc.append(epoch_valid_acc) # 검증 데이터셋을 모델에 적용했을 때의 '정확도'

end = time.time()
print(end-start)

# 실행 결과 보면, epoch=5 이므로, 정확도가 낮지만, 학습&검증 데이터셋에 대한 오차가 유사하기 때문에 과적합은 발생하지 않음을 알 수 있음

epoch:  0 loss：  0.011 accuracy: 0.494 valid_loss：  0.011 valid_accuracy: 0.507
epoch:  1 loss：  0.011 accuracy: 0.504 valid_loss：  0.011 valid_accuracy: 0.492
epoch:  2 loss：  0.011 accuracy: 0.512 valid_loss：  0.011 valid_accuracy: 0.496
epoch:  3 loss：  0.011 accuracy: 0.517 valid_loss：  0.011 valid_accuracy: 0.496
epoch:  4 loss：  0.011 accuracy: 0.525 valid_loss：  0.011 valid_accuracy: 0.511
6859.17232966423


In [28]:
# (코드 7-17) 모델 예측 함수 정의
def evaluate(epoch, model, testloader):    
    test_correct = 0
    test_total = 0
    test_running_loss = 0
    
    model.eval()
    with torch.no_grad():
        for b in testloader:
            x, y = b.text, b.label
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            y_pred = torch.argmax(y_pred, dim=1)
            test_correct += (y_pred == y).sum().item()
            test_total += y.size(0)
            test_running_loss += loss.item()

    epoch_test_loss = test_running_loss / len(testloader.dataset)
    epoch_test_acc = test_correct / test_total

    print('epoch: ', epoch,
          'test_loss: ', round(epoch_test_loss, 3),
          'test_accuracy:', round(epoch_test_acc, 3)
          )
    return epoch_test_loss, epoch_test_acc

In [29]:
# (코드 7-18) 모델 예측 결과 확인
epochs = 5
test_loss = []
test_acc = []

for epoch in range(epochs):
    epoch_test_loss, epoch_test_acc = evaluate(epoch, model, test_iterator)
    test_loss.append(epoch_test_loss) # 테스트 데이터셋을 모델에 적용했을 때의 '오차'
    test_acc.append(epoch_test_acc) # 테스트 데이터셋을 모델에 적용했을 때의 '정확도'

end = time.time()
print(end-start)

epoch:  0 test_loss:  0.011 test_accuracy: 0.5
epoch:  1 test_loss:  0.011 test_accuracy: 0.5
epoch:  2 test_loss:  0.011 test_accuracy: 0.5
epoch:  3 test_loss:  0.011 test_accuracy: 0.5
epoch:  4 test_loss:  0.011 test_accuracy: 0.5
6966.580687522888


### 7.4.2. RNN 계층 구현

In [30]:
# (코드 7-19) 라이브러리 호출
import torch
import torchtext
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import time

In [31]:
# (코드 7-20) IMDB 데이터셋을 내려받은 후 전처리 진행
from torchtext.legacy import datasets

start = time.time()
TEXT = torchtext.legacy.data.Field(sequential=True, batch_first=True, lower=True)
LABEL = torchtext.legacy.data.Field(sequential=False, batch_first=True)

# 학습 데이터, 테스트 데이터 분리
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
# 학습 데이터, 검증 데이터 분리
train_data, valid_data = train_data.split(split_ratio=0.8)

# 단어 집합 만들기 (중복은 제거된 상태에서 진행, 특정 단어가 최소 열 번 이상 등장한 것만 단어 집합에 추가)
TEXT.build_vocab(train_data, max_size = 10000, min_freq=10, vectors=None)
LABEL.build_vocab(train_data)

BATCH_SIZE = 100
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [33]:
# (코드 7-21) BucketIterator()를 이용하여 훈련/검증/테스트 데이터셋으로 분리
train_iterator, valid_iterator, test_iterator = torchtext.legacy.data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [34]:
# (코드 7-22) 변수 값 지정
# vocab_size: 영화 리뷰에 대한 텍스트 길이
# n_classes: 레이블(긍정, 부정) 값 지정
vocab_size = len(TEXT.vocab)
n_classes = 2 # pos(긍정), neg(부정)

In [35]:
# (코드 7-23) RNN 계층 네트워크
class BasicRNN(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p = 0.2):
        super(BasicRNN, self).__init__()
        self.n_layers = n_layers 
        self.embed = nn.Embedding(n_vocab, embed_dim) 
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)
        self.rnn = nn.RNN(embed_dim, self.hidden_dim, num_layers = self.n_layers, batch_first = True) 
        self.out = nn.Linear(self.hidden_dim, n_classes) 

    def forward(self, x):
        x = self.embed(x) 
        h_0 = self._init_state(batch_size = x.size(0)) 
        x, _ = self.rnn(x, h_0) 
        h_t = x[:, -1, :] 
        self.dropout(h_t)
        logit = torch.sigmoid(self.out(h_t))
        return logit

    def _init_state(self, batch_size = 1):
        weight = next(self.parameters()).data 
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [36]:
# (코드 7-24) 모델에서 사용할 손실함수, 옵티마이저 설정
model = BasicRNN(n_layers=1, hidden_dim=256, n_vocab=vocab_size, embed_dim=128, n_classes=n_classes, dropout_p=0.5)
model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [37]:
# (코드 7-25) 훈련 데이터셋을 이용해서 모델 학습시키기 위한 학습 (모델 학습 함수)
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(device), batch.label.to(device)
        y.data.sub_(1) # IMDB 레이블에서 긍정은 2, 부정은 1의 값을 갖음 >> y.data에서 1을 뺀다는 의미 = 레이블 값을 0과 1으로 변환하겠다는 의미
        optimizer.zero_grad()

        logit = model(x)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()

        if b % 50 == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(e, b * len(x), len(train_iter.dataset), 100. * b / len(train_iter), loss.item()))

In [38]:
# (코드 7-26) 모델 평가 함수
def evaluate(model, val_iter):
    model.eval()
    corrects, total, total_loss = 0, 0, 0

    for batch in val_iter:
        x, y = batch.text.to(device), batch.label.to(device)
        y.data.sub_(1) 
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction = "sum")
        total += y.size(0)
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
        
    avg_loss = total_loss / len(val_iter.dataset)
    avg_accuracy = corrects / total
    return avg_loss, avg_accuracy

In [39]:
# (코드 7-27) 훈련 데이터셋을 이용하여 모델을 학습 & 검증 데이터셋을 이용하여 성능(정확도) 확인
BATCH_SIZE = 100
LR = 0.001
EPOCHS = 5
for e in range(1, EPOCHS + 1):
    train(model, optimizer, train_iterator)
    val_loss, val_accuracy = evaluate(model, valid_iterator)
    print("[EPOCH: %d], Validation Loss: %5.2f | Validation Accuracy: %5.2f" % (e, val_loss, val_accuracy))

Train Epoch: 1 [0/20000 (0%)]	Loss: 0.714422
Train Epoch: 1 [5000/20000 (25%)]	Loss: 0.693294
Train Epoch: 1 [10000/20000 (50%)]	Loss: 0.691874
Train Epoch: 1 [15000/20000 (75%)]	Loss: 0.698530
[EPOCH: 1], Validation Loss:  0.69 | Validation Accuracy:  0.49
Train Epoch: 2 [0/20000 (0%)]	Loss: 0.694857
Train Epoch: 2 [5000/20000 (25%)]	Loss: 0.693424
Train Epoch: 2 [10000/20000 (50%)]	Loss: 0.692359
Train Epoch: 2 [15000/20000 (75%)]	Loss: 0.697260
[EPOCH: 2], Validation Loss:  0.69 | Validation Accuracy:  0.50
Train Epoch: 3 [0/20000 (0%)]	Loss: 0.695559
Train Epoch: 3 [5000/20000 (25%)]	Loss: 0.692803
Train Epoch: 3 [10000/20000 (50%)]	Loss: 0.693955
Train Epoch: 3 [15000/20000 (75%)]	Loss: 0.692833
[EPOCH: 3], Validation Loss:  0.69 | Validation Accuracy:  0.50
Train Epoch: 4 [0/20000 (0%)]	Loss: 0.689093
Train Epoch: 4 [5000/20000 (25%)]	Loss: 0.694507
Train Epoch: 4 [10000/20000 (50%)]	Loss: 0.692851
Train Epoch: 4 [15000/20000 (75%)]	Loss: 0.693906
[EPOCH: 4], Validation Loss:  0.

In [40]:
# (코드 7-28) 테스트 데이터셋을 이용한 모델 예측
test_loss, test_acc = evaluate(model,test_iterator) # (코드 7-26)에서 정의한 함수
print("Test Loss: %5.2f | Test Accuracy: %5.2f" % (test_loss, test_acc))

Test Loss:  0.70 | Test Accuracy:  0.44
